### 1. Setup Mario

In [ ]:
#Import the game
import gym_super_mario_bros
#import the Joypad wrapper
from nes_py.wrappers import JoypadSpace
#import the SIMPLIFIED controls
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

In [ ]:
#setup game
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)

In [ ]:
env.action_spacetion_space

In [ ]:
env.observation_space.shape

In [ ]:
#create a flag - restart or not

done = True
#loop through each frame in the game
for step in range(1000):
    if done:
    # start the game
        env.reset()
  # take random action
    state, reward, done, info = env.step(env.action_space.sample())
  # show the game on the screen
    env.render()
# close the game
env.close()

### 2. Preprocess Environment

In [ ]:
import os    
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [ ]:
#import Frame Stacker Wrapper and GrayScaling Wrapper
from gym.wrappers import GrayScaleObservation
#import Vectorization Wrappers
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
#import Matplotlib to show the impact of frame stacking
from matplotlib import pyplot as plt

In [ ]:
#1. create the base environment
env = gym_super_mario_bros.make('SuperMarioBros-v0')

#2. simplify the controls
env = JoypadSpace(env, SIMPLE_MOVEMENT)

#3. grayScale
env = GrayScaleObservation(env, keep_dim=True)

#4. wrap inside the dummy environment
env = DummyVecEnv([lambda: env])

#5. stack the frames
env = VecFrameStack(env, 4, channels_order='last')

In [ ]:
state = env.reset()

In [ ]:
state.shape

In [ ]:
state, reward, done, info = env.step([5])

In [ ]:
plt.figure(figsize=(20,16))
for idx in range(state.shape[3]):
    plt.subplot(1, 4, idx + 1)
    plt.imshow(state[0][:,:,idx])
plt.show()

### 3. Train the RL Model

In [ ]:
#import os for file path management
import os 
#import PPO for algos
from stable_baselines3 import PPO
#import Base Callback for saving models
from stable_baselines3.common.callbacks import BaseCallback

In [ ]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [ ]:
CHECKPOINT_DIR = './train/'
LOG_DIR = './logs/'

In [ ]:
#setup model saving callback
callback = TrainAndLoggingCallback(check_freq = 100000, save_path = CHECKPOINT_DIR)

In [ ]:
# ai model started
model = PPO('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001, n_steps=512)

In [ ]:
#train ai model 
model.learn(total_timesteps=1000000, callback=callback)

In [ ]:
model.save('my_model')

### 4. Test it Out

In [ ]:
model = PPO.load('./best_model_500000.zip')

In [ ]:

# Start the game 
state = env.reset()
# Loop through the game
while True: 
    
    action, _ = model.predict(state)
    state, reward, done, info = env.step(action)
    env.render()